In [36]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error, log_loss
pd.options.display.precision = 15
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
import time
import datetime
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import json
%matplotlib inline

%env JOBLIB_TEMP_FOLDER=/tmp

env: JOBLIB_TEMP_FOLDER=/tmp


In [37]:
out_path = './output/'
train_path = './data/original_train.csv'
test_path = './data/original_test_B.csv'

In [38]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
print(train.shape,test.shape)

(40000, 51) (12000, 50)


In [39]:
# 删除掉testb中与train分布不一致的特征列
need_drop_col = ['CUR_YEAR_MON_AGV_TRX_CNT','CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL','CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR','CUR_YEAR_MID_BUS_INC','AI_STAR_SCO',
'CUR_MON_COR_DPS_MON_DAY_AVG_BAL','ICO_CUR_MON_ACM_TRX_AMT','MON_12_AGV_TRX_CNT','MON_12_ACM_LVE_ACT_CNT','MON_12_AGV_LVE_ACT_CNT']

my_guess = [#
# 'MON_12_AGV_TRX_CNT',#1
# 'MON_12_ACM_ENTR_ACT_CNT',#0.98
# 'MON_12_AGV_ENTR_ACT_CNT',#0.98
# 'MON_12_ACM_LVE_ACT_CNT',#1
# 'MON_12_AGV_LVE_ACT_CNT'#1

#'LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL'
# 'PUB_TO_PRV_TRX_AMT_CUR_YEAR',#线下微掉
#'LAST_12_MON_COR_DPS_DAY_AVG_BAL', #据说线上掉的厉害，直接70，线下也是掉的厉害
#'NB_RCT_3_MON_LGN_TMS_AGV' #未验证，线下微涨
#'REG_DT', #注册日期
# 'REG_CPT',# 注册资本 这个个人觉得应该影响不大
#'OPN_TM' #开户日期

]

train = train.drop(need_drop_col + my_guess, axis=1)
test = test.drop(need_drop_col + my_guess, axis=1)
# train = train[['CUST_UID','LABEL'] + remain_col]
# test = test[['CUST_UID'] + remain_col]

print(train.shape,test.shape)



(40000, 41) (12000, 40)


In [40]:
# 分布修正
train['AGN_CUR_YEAR_AMT'] = train['AGN_CUR_YEAR_AMT'].apply(lambda x: np.log(x)-1.15)
train['AGN_CUR_YEAR_WAG_AMT'] = train['AGN_CUR_YEAR_WAG_AMT'].apply(lambda x: np.log(x)-1.15)

test['AGN_CUR_YEAR_AMT'] = test['AGN_CUR_YEAR_AMT'].apply(lambda x: np.log(x))
test['AGN_CUR_YEAR_WAG_AMT'] = test['AGN_CUR_YEAR_WAG_AMT'].apply(lambda x: np.log(x))

In [41]:
# 组合二阶特征
test['LABEL']=-1
data = pd.concat([train,test]).reset_index(drop=True)
print(data.shape)
original_fea = list(train.columns)[2:]
corr_fea = data[list(data.columns)].corr()
#list(corr_fea['AGN_CUR_YEAR_WAG_AMT'][corr_fea['AGN_CUR_YEAR_WAG_AMT'] > 0.3].index)
from tqdm import tqdm
cat_cols = ['MON_12_CUST_CNT_PTY_ID','AI_STAR_SCO','WTHR_OPN_ONL_ICO',
'SHH_BCK','LGP_HLD_CARD_LVL','NB_CTC_HLD_IDV_AIO_CARD_SITU'] #'COR_KEY_PROD_HLD_NBR',
history_data = []
very_low_corr_fea = []
print(len(list(corr_fea.columns)[2:]))
for col1 in tqdm(list(train.columns)[2:]):
    if col1 in cat_cols: continue
    # if len(list(corr_fea[col1][corr_fea[col1] > 0.3].index)) <2:
    #     very_low_corr_fea.append(col1)
    #print(len(list(corr_fea[col1][corr_fea[col1] > 0.3].index)))
    for col2 in list(corr_fea[col1][abs(corr_fea[col1] )> 0.3].index):
        if col1 == col2 or sorted([col1,col2]) in history_data: continue
        history_data.append(sorted([col1,col2]))
        train[f'corr_{col1}_{col2}_diff'] = train[col1] - train[col2]
        train[f'corr_{col1}_{col2}_rate'] = train[col1]/train[col2]
        train[f'corr_{col1}_{col2}_add'] = train[col1] + train[col2]
        train[f'corr_{col1}_rate'] = train[col1] /train[f'corr_{col1}_{col2}_add'] 
        train[f'corr_{col2}_rate'] = train[col2] /train[f'corr_{col1}_{col2}_add'] 

        test[f'corr_{col1}_{col2}_diff'] = test[col1] - test[col2]
        test[f'corr_{col1}_{col2}_rate'] = test[col1]/test[col2]
        test[f'corr_{col1}_{col2}_add'] = test[col1] + test[col2]
        test[f'corr_{col1}_rate'] = test[col1] /test[f'corr_{col1}_{col2}_add'] 
        test[f'corr_{col2}_rate'] = test[col2] /test[f'corr_{col1}_{col2}_add']
print(len(history_data))
    

(52000, 41)


 26%|██▌       | 10/39 [00:00<00:00, 78.47it/s]

34


100%|██████████| 39/39 [00:00<00:00, 88.03it/s]

62


In [42]:
# 手工特征
def check_duplic_cols(feature,his_features,sorte_flag = True):
    if sorte_flag:
        if sorted(feature) in his_features:
            return True
        else:
            return False
    else:
        if feature in his_features:
            return True
        else:
            return False
def gen_diff_fea(df, features,his_features = []):
    try:
        for fea in features:
            if check_duplic_cols(fea,his_features): continue
            df[f'hand_{fea[0]}_{fea[1]}_diff'] = train[fea[0]] - train[fea[1]]
            his_features.append(sorted(fea))
        return df
    except:
        return df
def gen_rate_fea(df,features,his_features = []):
    try:
        for fea in features:
            if check_duplic_cols(fea,his_features,sorte_flag =False): continue
            df[f'hand_{fea[0]}_{fea[1]}_rate'] = train[fea[0]]/train[fea[1]]
            his_features.append(fea)
        return df
    except:
        return df

rate_features = [['MON_12_ACT_IN_50_UP_CNT_PTY_QTY','MON_12_ACT_OUT_50_UP_CNT_PTY_QTY'],['MON_12_EXT_SAM_TRSF_IN_AMT','MON_12_EXT_SAM_TRSF_OUT_AMT'],
['CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT','CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT'],['MON_6_50_UP_ENTR_ACT_CNT','MON_6_50_UP_LVE_ACT_CNT'],['MON_12_EXT_SAM_AMT','MON_12_EXT_SAM_NM_TRSF_OUT_CNT']]
train = gen_rate_fea(train, rate_features ,his_features = [])
test = gen_rate_fea(test, rate_features ,his_features = [])

diff_features = [['MON_12_EXT_SAM_AMT','MON_12_EXT_SAM_NM_TRSF_OUT_CNT']]
train = gen_diff_fea(train, diff_features ,his_features = [])
test = gen_diff_fea(test, diff_features ,his_features = [])


In [43]:
# 删除重复特征
print(train.shape,test.shape)
corr_fea = train[list(train.columns)].corr()
#list(corr_fea['AGN_CUR_YEAR_WAG_AMT'][corr_fea['AGN_CUR_YEAR_WAG_AMT'] > 0.3].index)
from tqdm import tqdm
cat_cols = ['MON_12_CUST_CNT_PTY_ID','AI_STAR_SCO','WTHR_OPN_ONL_ICO',
'SHH_BCK','LGP_HLD_CARD_LVL','NB_CTC_HLD_IDV_AIO_CARD_SITU'] #'COR_KEY_PROD_HLD_NBR',

drop_data = ['CUR_YEAR_MON_AGV_']
very_low_corr_fea = []
print(len(list(corr_fea.columns)[2:]))
#print(corr_fea)
for col1 in tqdm(list(train.columns)[2:]):
    if col1 in cat_cols: continue
    # if len(list(corr_fea[col1][corr_fea[col1] > 0.3].index)) <2:
    #     very_low_corr_fea.append(col1)
    #print(len(list(corr_fea[col1][corr_fea[col1] > 0.3].index)))
    for col2 in list(corr_fea[col1][corr_fea[col1] == 1].index):
        if col1 == col2 or sorted([col1,col2]) in drop_data: continue
        drop_data.append(sorted([col1,col2]))
for col in drop_data:
    if (col[0] in list(train.columns)) and (col[1] in list(train.columns)):
        train = train.drop([col[0]], axis=1)
        test = test.drop([col[0]], axis=1)
       

print(len(drop_data))
print(train.shape,test.shape)

(40000, 259) (12000, 259)


100%|██████████| 257/257 [00:00<00:00, 4450.55it/s]

252


5
(40000, 255) (12000, 255)


In [44]:
train.to_csv('./data/train_fea_genb4.csv',index=None)
test.to_csv('./data/test_B_fea_genb4.csv',index=None)